# Turn Videos into ASCII Art Videos
As Seen On: [Instructables by Tom Savoie](https://www.instructables.com/Turn-Videos-Into-ASCII-Art-Videos/)


In [ ]:
from PIL import Image,ImageOps,ImageEnhance
import cv2
import os
import imgkit

## Seperate video frame-by-frame and save as jpg

### Args:
> path (string): path to video

### Returns:
> fps: the video speed
> counter-1: total amount of images

In [ ]:
def video_to_images(path):
    os.mkdir('Images')
    video = cv2.VideoCapture(path)
    fps = video.get(cv2.CAP_PROP_FPS)
    success, image = video.read()
    counter = 1
    while success:
        cv2.imwrite("Images/Image{0}.jpg".format(str(counter)), image)
        success, image = video.read()
        counter += 1
    return fps, (counter-1)


## Flatten the Image

### Args:
> image_path (sting): path to image

### Returns:
> initial_img: Flattened Image

In [ ]:
def get_image(image_path):
    initial_image = Image.open(image_path)
    # save dimensions
    width, height = initial_image.size
    # resize with larger width to that its normal when ascii
    initial_image = initial_image.resize((round(width*1.05), height))
    return initial_image

## Resize image to set ratio

### Args:
> image ([type]): working image
> final_width (image Width, optional): Width we resize it by. Defaults to 200.

### Returns:
> image: Resized image


In [ ]:
def pixelate_image(image, final_width=200):
    width, height = image.size
    # calc corresponding height
    final_height = int((height*final_width)/width)
    image = image.resize((final_width, final_height))
    # Only if too dim
    #image = ImageEnhance.Brightness(image)
    #image = image.enhance(1.5)
    return image


## Greyscale image

ImageOps.grayscale() is a PIL method to render an image in grey values

In [ ]:
def grayscale_image(image):
    image_bw = ImageOps.grayscale(image)
    return image_bw

## Convert BnW Image into Opacity value depending on char list

### Args:
> bw_image ([type]): the BnW image
> ASCII_STR (list): the ASCII Values

### Returns:
> image list: Stock of chars for image

#### ascii_conv:: 
With a cross-multiplication we get a value between 0 and the number of characters in the ASCII_STR list minus 1 (because it starts counting from 0)
corresponding to the RGB value of it. In this example, a black will correspond to 0 and a white pixel to 8.

#### append :: 
From this value between 0 and 8 (if the length of the string is 9) we will add in our ASCII characters list, the character corresponding to the opacity of the pixel. Here, black corresponds to " " and white to "&".


In [ ]:
def ascii_conversion(bw_image, ascii_string=[" ", ".", ":", "-", "=", "+", "*", "#", "%", "@", "&"]):
    pixels = bw_image.getdata()
    ascii_image_list = []
    
    for pixel in pixels:
        ascii_converted = int((pixel*len(ascii_string))/256)
        ascii_image_list.append(ascii_string[ascii_converted])
    return ascii_image_list

## Get Colour
Take away RGB Values of each pixel to recolour later

In [ ]:
def get_color(image):
    pixels = image.getdata()  
    # Creates a list with the RGB value for each pixel
    return pixels

## Store ASCII Version
Create an HTML Document to store the ascii version
### Requires
> The list of the ASCII characters
> The pixelated image (to retrieve its size)
> The color list created in STEP 6
> The position of the image in the video (to name it and to distinguish it from the others)

In [ ]:
def print_ascii(ascii_list, image, color, image_pos):
    file = open('HtmlImages/Html{0}.html'.format(str(image_pos)), "w")
    file.write("""                                      
     <!DOCTYPE html>
     <html>
        <body style='background-color:black'>
        <pre style='display: inline-block; border-width: 4px 6px; border-color: black; border-style: solid; background-color:black; font-size: 32px ;font-face: Montserrat;font-weight: bold;line-height:60%'>""")

    width, height = image.size
    counter = 0
    for j in ascii_list:
        color_hex = '%02x%02x%02x' % color[counter]
        counter += 1
        if (counter % width) != 0:
            file.write(
                "<span style=\"color: #{0}\">{1}</span>".format(color_hex, j))
        else:
            file.write("<br />")
    file.write("""</pre></body>
     </html>""")
    file.close()

## Main *Madgik*

In [ ]:
def main(video_path):
    #config = imgkit.config(wkhtmltoimage=r'wkhtmltoimage')
    ascii_string = [" ", "|", "/",
                    ".", ":", "-", 
                    "=", "+", "*",
                    "#", "%", "@", "&"]
    fps, number_images = video_to_images(video_path)
    os.mkdir('HtmlImages')
    os.mkdir('TextImages')

    for i in range(1, number_images+1):
        image = get_image('Images/Image{0}.jpg'.format(str(i)))
        right_size_image = pixelate_image(image)
        bw_image = grayscale_image(right_size_image)
        converted_list = ascii_conversion(bw_image, ascii_string)
        color_list = get_color(right_size_image)
        print_ascii(converted_list, right_size_image, color_list, i)
        imgkit.from_file('HtmlImages/Html{0}.html'.format(str(i)),
                         'TextImages/Image{0}.jpg'.format(str(i)))

    res = Image.open('TextImages/Image1.jpg').size
    video = cv2.VideoWriter('final_video.mp4', cv2.VideoWriter_fourcc(
        'm', 'p', '4', 'v'), int(fps), res)

    for j in range(1, number_images+1):
        video.write(cv2.imread('TextImages/Image{0}.jpg'.format(str(j))))
    video.release()


## Supply Video

In [42]:
if __name__ == "__main__":
    main("Car.mp4")